### 모델 정의

Optuna를 이용하여 파라미터를 구함

In [59]:
# LightGBM_dart
lgbm_dart_model = LGBMClassifier(
    n_estimators=1029
    , num_leaves=167
    , max_depth=30
    , learning_rate=0.05767571715999541
    , min_child_samples=25
    , verbose=-1
    , boosting='dart'  # dart 사용
    , random_state=0
)

lgbm_dart_model.fit(x_train, y_train)

pred = lgbm_dart_model.predict(x_val)
get_clf_eval(y_val, pred)

# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

test_pred = lgbm_dart_model.predict(x_test)
sum(test_pred) # True로 예측된 개수


오차행렬:


,True,False
True,830,164
False,154,10712


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.973187,0.843496,0.83501,0.839232,0.973126


799.0

Public : 0.54688731

------

In [60]:
# XGBoost 
xgb_model = XGBClassifier(
    n_estimators=414
    , learning_rate=0.20046808426888615
    , max_depth=11

    , alpha=0.004365542651458743
    , gamma=0.00025712949731685885

    , reg_alpha=0.17168922089033928
    , reg_lambda=0.03881395024846057
    
    , colsample_bytree=0.32031741412326675
    , subsample=0.6269215430592496
    , objective='binary:logistic'  # 이진 분류
    , tree_method="exact"        
    , random_state=0
)

xgb_model.fit(x_train, y_train)

pred = xgb_model.predict(x_val)
get_clf_eval(y_val, pred)

# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

test_pred = xgb_model.predict(x_test)
sum(test_pred) # True로 예측된 개수


오차행렬:


,True,False
True,817,177
False,126,10740


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.974452,0.866384,0.821932,0.843573,0.974146


872

Public : 0.58030303

------

In [61]:
from sklearn.ensemble import VotingClassifier

# LightGBM_dart
lgbm_dart_model = LGBMClassifier(
    n_estimators=1029
    , num_leaves=167
    , max_depth=30
    , learning_rate=0.05767571715999541
    , min_child_samples=25
    , verbose=-1
    , boosting='dart'  # dart 사용
    , random_state=0
)

# XGBoost 
xgb_model = XGBClassifier(
    n_estimators=414
    , learning_rate=0.20046808426888615
    , max_depth=11

    , alpha=0.004365542651458743
    , gamma=0.00025712949731685885

    , reg_alpha=0.17168922089033928
    , reg_lambda=0.03881395024846057
    
    , colsample_bytree=0.32031741412326675
    , subsample=0.6269215430592496
    , objective='binary:logistic'  # 이진 분류
    , tree_method="exact"        
    , random_state=0
)

# ExtraTrees
et_model = ExtraTreesClassifier(
    n_estimators=100
    , max_depth=48
    , min_samples_split=3
    , min_samples_leaf=1
    , criterion='gini'
    , random_state=0
)

# 보팅 분류기 생성
voting_model = VotingClassifier(
    estimators=[
        ('lgb_dart', lgbm_dart_model),
        ('xgb', xgb_model),
        ('et', et_model)
    ],
    voting='soft'  # 'hard'는 다수결 투표, 'soft'는 확률 평균
)


voting_model.fit(x_train, y_train)

VotingClassifier(estimators=[('lgb_dart',
                              LGBMClassifier(boosting='dart',
                                             learning_rate=0.05767571715999541,
                                             max_depth=30, min_child_samples=25,
                                             n_estimators=1029, num_leaves=167,
                                             random_state=0, verbose=-1)),
                             ('xgb',
                              XGBClassifier(alpha=0.004365542651458743,
                                            base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_by...
                                            learning_rate=0.20046808426888615,
                                            max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=11,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=414, n_jobs=None,
                                            num_parallel_tree=None, ...)),
                             ('et',
                              ExtraTreesClassifier(max_depth=48,
                                                   min_samples_split=3,
                                                   random_state=0))],
                 voting='soft')

In [62]:
# 확률 예측
soft_voting_probs = voting_model.predict_proba(x_val)[:, 1]

In [63]:
# 스레시홀드 0.4 적용
soft_voting_preds_04 = [1 if prob > 0.4 else 0 for prob in soft_voting_probs]

# 평가
get_clf_eval(y_val, soft_voting_preds_04)

# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

# 확률 예측
test_pred_probs = voting_model.predict_proba(x_test)[:, 1]

# 스레시홀드 적용
preds_04 = [1 if prob > 0.4 else 0 for prob in test_pred_probs]

sum(preds_04) # True로 예측된 개수


오차행렬:


,True,False
True,843,151
False,170,10696


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.972934,0.832182,0.848089,0.84006,0.973051


967

Public : 0.618556701

----

In [64]:
# 스레시홀드 0.2 적용
soft_voting_preds_02 = [1 if prob > 0.2 else 0 for prob in soft_voting_probs]

get_clf_eval(y_val, soft_voting_preds_02)

# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

# 확률 예측
test_pred_probs = voting_model.predict_proba(x_test)[:, 1]

# 스레시홀드 적용
preds_02 = [1 if prob > 0.2 else 0 for prob in test_pred_probs]

sum(preds_02) # True로 예측된 개수


오차행렬:


,True,False
True,900,94
False,301,10565


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.966695,0.749376,0.905433,0.820046,0.968105


1408

Public : 0.70513633481

-----

In [65]:
# 스레시홀드 0.1 적용
soft_voting_preds_02 = [1 if prob > 0.1 else 0 for prob in soft_voting_probs]

get_clf_eval(y_val, soft_voting_preds_02)

# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

# 확률 예측
test_pred_probs = voting_model.predict_proba(x_test)[:, 1]

# 스레시홀드 적용
preds_02 = [1 if prob > 0.1 else 0 for prob in test_pred_probs]

sum(preds_02) # True로 예측된 개수


오차행렬:


,True,False
True,942,52
False,486,10380


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.954637,0.659664,0.947686,0.77787,0.95824


1917

Public : 0.751375137

----

In [66]:
# 스레시홀드 적용
preds_01 = [1 if prob > 0.08 else 0 for prob in soft_voting_probs]

# 평가
get_clf_eval(y_val, preds_01)

# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

# 확률 예측
test_pred_probs = voting_model.predict_proba(x_test)[:, 1]

# 스레시홀드 적용
test_pred = [1 if prob > 0.08 else 0 for prob in test_pred_probs]

sum(test_pred) # True로 예측된 개수


오차행렬:


,True,False
True,952,42
False,559,10307


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.949325,0.630046,0.957746,0.76008,0.953937


2063

Public : 0.753577106

----

In [92]:
# LightGBM_dart
lgbm_dart_model = LGBMClassifier(
    n_estimators=1029
    , num_leaves=167
    , max_depth=30
    , learning_rate=0.05767571715999541
    , min_child_samples=25
    , verbose=-1
    , boosting='dart'  # dart 사용
    , random_state=0
)

# XGBoost 
xgb_model = XGBClassifier(
    n_estimators=414
    , learning_rate=0.20046808426888615
    , max_depth=11
    , alpha=0.004365542651458743
    , gamma=0.00025712949731685885
    , reg_alpha=0.17168922089033928
    , reg_lambda=0.03881395024846057
    , colsample_bytree=0.32031741412326675
    , subsample=0.6269215430592496
    , objective='binary:logistic'  # 이진 분류
    , tree_method="exact"        
    , random_state=0
)

# DecisionTree
dt_model = DecisionTreeClassifier(
    max_depth=10
    , min_samples_split=2
    , min_samples_leaf=1
    , max_features=None
    , splitter='best'
    , criterion='gini'
    , random_state=0
)  

# ExtraTrees
et_model = ExtraTreesClassifier(
    n_estimators=100
    , max_depth=48
    , min_samples_split=3
	, min_samples_leaf=1
	, criterion='gini'
	, random_state=0
)

### 보팅 분류기 생성 ###
voting_model_ver2 = VotingClassifier(
    estimators=[
        ('lgb_dart',lgbm_dart_model),
        ('xgb', xgb_model),
        ('dt', dt_model),
        ('et', et_model)
    ],
    voting='soft'  # 'hard'는 다수결 투표, 'soft'는 확률 평균
)

voting_model_ver2.fit(x_train, y_train)

VotingClassifier(estimators=[('lgb_dart',
                              LGBMClassifier(boosting='dart',
                                             learning_rate=0.05767571715999541,
                                             max_depth=30, min_child_samples=25,
                                             n_estimators=1029, num_leaves=167,
                                             random_state=0, verbose=-1)),
                             ('xgb',
                              XGBClassifier(alpha=0.004365542651458743,
                                            base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_by...
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=11,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=414, n_jobs=None,
                                            num_parallel_tree=None, ...)),
                             ('dt',
                              DecisionTreeClassifier(max_depth=10,
                                                     random_state=0)),
                             ('et',
                              ExtraTreesClassifier(max_depth=48,
                                                   min_samples_split=3,
                                                   random_state=0))],
                 voting='soft')

In [93]:
# 확률 예측
soft_voting_probs = voting_model_ver2.predict_proba(x_val)[:, 1]

In [95]:
# 스레시홀드 0.5 적용
soft_voting_preds_05 = [1 if prob > 0.5 else 0 for prob in soft_voting_probs]

get_clf_eval(y_val, soft_voting_preds_05)

# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

# 확률 예측
test_pred_probs = voting_model_ver2.predict_proba(x_test)[:, 1]

# 스레시홀드 적용
preds_02 = [1 if prob > 0.5 else 0 for prob in test_pred_probs]

sum(preds_02) # True로 예측된 개수


오차행렬:


,True,False
True,843,151
False,168,10698


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.973103,0.833828,0.848089,0.840898,0.973206


908

Public : 0.68

----

In [96]:
# 스레시홀드 0.2 적용
soft_voting_preds_02 = [1 if prob > 0.2 else 0 for prob in soft_voting_probs]

get_clf_eval(y_val, soft_voting_preds_02)

# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

# 확률 예측
test_pred_probs = voting_model_ver2.predict_proba(x_test)[:, 1]

# 스레시홀드 적용
preds_02 = [1 if prob > 0.2 else 0 for prob in test_pred_probs]

sum(preds_02) # True로 예측된 개수


오차행렬:


,True,False
True,945,49
False,552,10314


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.949325,0.631263,0.950704,0.758731,0.953841


1827

Public : 0.74455002

----

In [67]:
## 이전 하이퍼 파라미터에서 LightGBM_dart 만 수정하고 
## 나머지 조합등의 사항은 유지함
# LightGBM_dart 
lgbm_dart_model = LGBMClassifier(
    n_estimators=716
    , num_leaves=677
    , max_depth=24
    , learning_rate=0.09716914697888186
    , min_child_samples=12
    , verbose=-1
    , boosting='dart'  # dart 사용
    , random_state=0
)

# XGBoost 
xgb_model = XGBClassifier(
    n_estimators=414
    , learning_rate=0.20046808426888615
    , max_depth=11

    , alpha=0.004365542651458743
    , gamma=0.00025712949731685885

    , reg_alpha=0.17168922089033928
    , reg_lambda=0.03881395024846057
    
    , colsample_bytree=0.32031741412326675
    , subsample=0.6269215430592496
    , objective='binary:logistic'  # 이진 분류
    , tree_method="exact"        
    , random_state=0
)

# ExtraTrees
et_model = ExtraTreesClassifier(
    n_estimators=100
    , max_depth=48
    , min_samples_split=3
    , min_samples_leaf=1
    , criterion='gini'
    , random_state=0
)

### 보팅 분류기 생성 ###
voting_model = VotingClassifier(
    estimators=[
        ('lgbm_dart',lgbm_dart_model),
        ('xgb', xgb_model),
        ('et', et_model)
    ],
    voting='soft'  # 'hard'는 다수결 투표, 'soft'는 확률 평균
)

voting_model.fit(x_train, y_train)

VotingClassifier(estimators=[('lgbm_dart',
                              LGBMClassifier(boosting='dart',
                                             learning_rate=0.09716914697888186,
                                             max_depth=24, min_child_samples=12,
                                             n_estimators=716, num_leaves=677,
                                             random_state=0, verbose=-1)),
                             ('xgb',
                              XGBClassifier(alpha=0.004365542651458743,
                                            base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_by...
                                            learning_rate=0.20046808426888615,
                                            max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=11,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=414, n_jobs=None,
                                            num_parallel_tree=None, ...)),
                             ('et',
                              ExtraTreesClassifier(max_depth=48,
                                                   min_samples_split=3,
                                                   random_state=0))],
                 voting='soft')

In [68]:
# 확률 예측
soft_voting_probs = voting_model.predict_proba(x_val)[:, 1]

In [69]:
# 스레시홀드 적용
soft_voting_preds = [1 if prob > 0.1 else 0 for prob in soft_voting_probs]

get_clf_eval(y_val, soft_voting_preds)

# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

# 확률 예측
test_pred_probs = voting_model.predict_proba(x_test)[:, 1]

# 스레시홀드 적용
preds = [1 if prob > 0.1 else 0 for prob in test_pred_probs]

sum(preds) # True로 예측된 개수


오차행렬:


,True,False
True,929,65
False,446,10420


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.956914,0.675636,0.934608,0.784297,0.959994


1815

Public : 0.74760022

----

In [70]:

from sklearn.ensemble import VotingClassifier

# LightGBM_dart
lgbm_dart_model = LGBMClassifier(
    n_estimators=1029
    , num_leaves=167
    , max_depth=30
    , learning_rate=0.05767571715999541
    , min_child_samples=25
    , verbose=-1
    , boosting='dart'  # dart 사용
    , random_state=0
)

# XGBoost 
xgb_model = XGBClassifier(
    n_estimators=414
    , learning_rate=0.20046808426888615
    , max_depth=11

    , alpha=0.004365542651458743
    , gamma=0.00025712949731685885

    , reg_alpha=0.17168922089033928
    , reg_lambda=0.03881395024846057
    
    , colsample_bytree=0.32031741412326675
    , subsample=0.6269215430592496
    , objective='binary:logistic'  # 이진 분류
    , tree_method="exact"        
    , random_state=0
)

# CatBoost
cat_model = CatBoostClassifier(
    iterations=627
    , depth=8
    , learning_rate=0.16672163155553238
    , l2_leaf_reg= 0.019556805887189933
    , border_count = 163
    , verbose=False
    , random_state=0
)

# ExtraTrees
et_model = ExtraTreesClassifier(
    n_estimators=100
    , max_depth=48
    , min_samples_split=3
    , min_samples_leaf=1
    , criterion='gini'
    , random_state=0
)

### 보팅 분류기 생성 ###
voting_model = VotingClassifier(
    estimators=[
        ('lgbm_dart',lgbm_dart_model),
        ('xgb', xgb_model),
        ('cat', cat_model),
        ('et', et_model)
    ],
    voting='soft'  # 'hard'는 다수결 투표, 'soft'는 확률 평균
)

voting_model.fit(x_train, y_train)

VotingClassifier(estimators=[('lgbm_dart',
                              LGBMClassifier(boosting='dart',
                                             learning_rate=0.05767571715999541,
                                             max_depth=30, min_child_samples=25,
                                             n_estimators=1029, num_leaves=167,
                                             random_state=0, verbose=-1)),
                             ('xgb',
                              XGBClassifier(alpha=0.004365542651458743,
                                            base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_b...
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=11,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=414, n_jobs=None,
                                            num_parallel_tree=None, ...)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x0000019599B1F4C0>),
                             ('et',
                              ExtraTreesClassifier(max_depth=48,
                                                   min_samples_split=3,
                                                   random_state=0))],
                 voting='soft')

In [71]:
# 확률 예측
soft_voting_probs = voting_model.predict_proba(x_val)[:, 1]

In [72]:
# 스레시홀드 적용
soft_voting_preds = [1 if prob > 0.1 else 0 for prob in soft_voting_probs]

get_clf_eval(y_val, soft_voting_preds)

# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

# 확률 예측
test_pred_probs = voting_model.predict_proba(x_test)[:, 1]

# 스레시홀드 적용
preds = [1 if prob > 0.1 else 0 for prob in test_pred_probs]

sum(preds) # True로 예측된 개수


오차행렬:


,True,False
True,934,60
False,450,10416


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.956998,0.674855,0.939638,0.785534,0.960132


1774

Public : 0.746712407089

----

.